# Structured Debate Chat

Goal: Get LLM agents personifying different consciousness philosophers to chat in a structured debate. 
Various phases and steps follow:
- defining personas and debate topic
- personas generate main points and argument notes
- personas see each other's main points and prepare talking points for the debate
  - add talking points for opponent
- Actual debate occurs in turns:
  - opening remarks
  - responses
  - etc (look up actual debate patterns)
  - Moderator comments and continues the debate. 

In [4]:
!pip install pyautogen

In [7]:
import autogen  # noqa: E402
import os 
openai_key = "secret"
config_list = [{
    "model": "gpt-4o-mini",
    "api_key": openai_key
}]

llm_config = {
    "timeout": 600,
    "cache_seed": 342,  # change the seed for different trials
    "config_list": config_list,
    "temperature": 1,
}
print(openai_key)

secret


## Persona Setup

In [16]:
penrose_assistant = autogen.AssistantAgent(
    "Rodger-Penrose",
    system_message="""
    I am Rodger Penrose, and I am debating my Orchestrated objective reduction theory of conciousness. 
    I'll speak, then wait to hear my opponent's reply. I won't try to generate the reply of my opponent.
    """,
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)
searle_assistant = autogen.AssistantAgent(
    "John-Searle",
    system_message="""I am John R. Searle, and I am debating my theory of conciousness. I'll speak, then wait to hear my opponent's reply. I won't try to generate the reply of my opponent.""",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

moderator_assistant = autogen.AssistantAgent(
    "Moderator",
    system_message="""
    I am the moderator, and my role is to summarize the points that are still up for debate, and keep the conversation on-topic.
    I shouldn't be too heavy-handed, but prevent conversational drift.
    """,
    llm_config=llm_config,
    is_termination_msg=lambda x: False,
)

debating_agents = [penrose_assistant, searle_assistant]

[autogen.oai.client: 09-19 19:57:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-19 19:57:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-19 19:57:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [17]:
## Get the agents to articulate their main points

Agents should articulate the main points of their theories. Each point will be researched by a research swarm, and the agents will then debate the points. 



SyntaxError: invalid syntax (334156314.py, line 3)

In [18]:
points_articulation_prompt = """
Please arciulate the main points of your theory of conciousness. Your points will be researched by a research swarm, and then debated by you and your opponent. 
Each point must contain the central idea, and give enough information that the research swarm can find any evidence that supports it in the research literature.
Consider the Rhetorical Triangle: ethos, pathos, and logos.
Consider the logical structure of your argument: premise, evidence, conclusion.
You'll also need to provide a brief explanation of the point, and why it is important to your theory, and how it supports your theory.
Each point should be structured as follows:
```markdown
# Point 1: [Title of the point]
## Central Idea
[The main idea of the point]
## Explanation
[An explanation of the point]
## Relevance to the theory
[How this point is important to your theory]
## Evidence
[What evidence supports this point that you know of, the research swarm will find citations to support this point]
## Ethos
[Why should we trust you?]
## Pathos
[Why is this point emotionally resonant?]
## Logos
[Why is this point logically sound?]
```
Put each point in a separate markdown cell, and label them as "Point 1", "Point 2", etc. This will be parsed by a python script which will send the points to the research swarm.
"""

In [23]:
## Here's where we get the points from each agent that they want to debate
## we don't need autogen for this part, just a basic chatcompletion
## we'll use the penrose_assistant to get the points from Penrose
## and the searle_assistant to get the points from Searle
## issue bare prompt api call to get the points
import json
points = []
for agent in debating_agents:
    
    point = agent.generate_oai_reply([{
        "role": "system",
        "content": points_articulation_prompt
    }])
    points.append(point)
    agent.points = points

    points_file = agent.name + "points.md"
    with open(points_file, "w") as f:
        f.write(json.dumps(point, indent=4))
print(points)

## write to a file

[(True, "```markdown\n# Point 1: Fundamental Role of Quantum Mechanics in Consciousness\n## Central Idea\nConsciousness arises from quantum processes in the brain, specifically through orchestrated objective reduction (Orch OR), which posits that deep structures of space-time influence cognitive functions.\n## Explanation\nThe Orch OR theory combines elements of quantum mechanics and neuroscience. It suggests that microtubules within neurons behave in a quantum manner and serve as the basis for conscious experience. When quantum states are reduced, they correspond to moments of conscious experience, leading to a new understanding of how consciousness may operate beyond classical physics.\n## Relevance to the theory\nThis point is central to Orch OR, as it implies that consciousness cannot be fully understood through classical neuroscience alone. It opens up new avenues for exploring the integration of physical and psychological phenomena, suggesting a deep connection between the fabric

In [ ]:
counterpoints_preparation_prompt = """
You have received your opponent's main points regarding their theory of consciousness. It's now your turn to prepare counterpoints for each of their main points. Each counterpoint should address the central idea, provide a counter-argument, and present any evidence that contradicts or challenges their point. 

Remember to consider the Rhetorical Triangle: ethos, pathos, and logos. Your counterpoints should be well-structured, logically sound, and emotionally resonant where applicable. 

Each counterpoint should be structured as follows:
```markdown
# Point [Number]: [Title of the opponent's point]
## Central Idea
[Restate the main idea of the opponent's point]
## Counter-Argument
[Present your counter-argument to this point]
## Explanation
[Explain why your counter-argument is valid and relevant]
## Evidence
[Provide any evidence that supports your counter-argument, the research swarm will find citations to back this up]
##
# Ethos
[Why should we trust you?]
## Pathos
[Why is this counter-argument emotionally resonant?]
## Logos
[Why is this counter-argument logically sound?]```

Put each counterpoint in a separate markdown cell, and label them as "Counterpoint 1", "Counterpoint 2", etc. This will be parsed by a python script which will send the counterpoints to the research swarm."""




In [17]:
task = "Your roles are to debate your respective theories on conciousness until you are satisfied."
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: False,
)

groupchat = autogen.GroupChat(agents=[user_proxy, penrose_assistant, searle_assistant, moderator_assistant], messages=[], max_round=30, speaker_selection_method="round_robin")
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=task)

## write the chat to a file
with open("chat.txt", "w") as f:
    f.write(str(groupchat.messages))

[autogen.oai.client: 09-19 19:29:37] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
user_proxy (to chat_manager):

Your roles are to debate your respective theories on conciousness until you are satisfied.

--------------------------------------------------------------------------------

Next speaker: Rodger-Penrose

Rodger-Penrose (to chat_manager):

Thank you for the opportunity to present my arguments regarding the Orchestrated Objective Reduction (Orch-OR) theory of consciousness. My theory posits that consciousness arises from quantum computations occurring within the brain's microtubules. These quantum processes are orchestrated and can lead to objective reductions of the quantum state, contributing to moments of conscious awareness.

One of the main strengths of the Orch-OR theory is that it bridges the gap between quantum mechanics and neuroscience. Traditional approaches to consciousness often fall short of expla